Here basically, we have to find probablities of each class(1-9) for each id(0-3320) of gene and class having 
maximum probablity for a particular gene will be declared as the class of that gene.

For test

input---->id,gene,variation,text documentation\
output--->class

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
variant = pd.read_csv(r"/kaggle/input/msk-redefining-cancer-treatment/training_variants.zip")

In [ ]:
# as this is non comma seperated values so some changes in it

text_data = pd.read_csv(r"/kaggle/input/msk-redefining-cancer-treatment/training_text.zip" ,sep="\|\|",engine="python",names=["ID","TEXT"],skiprows=1)

In [ ]:
text_data

### Pre Processing text data

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [ ]:
print(stop_words)

In [ ]:
import re

def cleaning(text, index, column):
    if type(text) is not int:
        
        string = ""
        # replace every special char with space
        text = re.sub('[^a-zA-Z0-9\n]', ' ', text)
        
        # replace multiple spaces with single space
        text = re.sub('\s+',' ', text)
        
        # converting all the chars into lower-case.
        text = text.lower()
        
        for word in text.split():
            if word not in stop_words:
                string += word + " "
        
        text_data[column][index] = string
                
                
                

In [ ]:
for index, row in text_data.iterrows():
    if type(row['TEXT'] ) is str:
        cleaning(row['TEXT'], index , 'TEXT')
    else:
        print("there is no text description for id:",index)

In [ ]:
#merging both gene_variations and text data based on ID

result = pd.merge(variant, text_data,on='ID', how='left')
result.head()

In [ ]:
result[result.isnull().any(axis = 1)]

In [ ]:
result.loc[result["TEXT"].isnull(),'TEXT'] = result["Gene"] + "" + result["Variation"]

In [ ]:
result[result["ID"] == 1109]

### Splitting data

In [ ]:
result

In [ ]:
y_true = result['Class'].values
result.Gene      = result.Gene.str.replace('\s+', '_')
result.Variation = result.Variation.str.replace('\s+', '_')

In [ ]:
result.head()

In [ ]:
x = result.drop("Class" ,axis = 1)
y = result.iloc[: ,3:4]

In [ ]:
from sklearn.model_selection import train_test_split

x_tr, x_test, y_tr, y_test = train_test_split(x, y, test_size=0.2 ,random_state = 0)
x_train, x_cv, y_train, y_cv = train_test_split(x_tr, y_tr,test_size=0.2 ,random_state = 0)

In [ ]:
print(x_tr.shape)
print(x_test.shape)
print(y_tr.shape)
print(y_test.shape)

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_cv.shape)
print(y_cv.shape)

In [ ]:
print("total training data = " , x_train.shape[0])
print("total cross validating data = " , x_cv.shape[0])
print("total testing data = " , x_test.shape[0])

### Some visualization

In [ ]:
train_class_distribution = y_train['Class'].value_counts()
train_class_distribution.plot(kind = 'bar')
plt.grid()

In [ ]:
test_class_distribution = y_test['Class'].value_counts()
test_class_distribution.plot(kind = 'bar')

By above both plots we can understand that data is highly unbalanced and also both train\
and test are in sync(according to classes)

### Prediction using a 'Random' Model</h2>

In [ ]:
'''in this what we are doing as it is told that we have to use log loss for comparing and the thing is while 
using auc we know that for random numbers auc is 0.5 so we have to make our model with accuracy more than 0.5
same thing here as we are using log loss so first we will determine log loss for random numbers with this datasets
and then we will try to built our model with log loss less than that random log loss'''

'''we will take probablities of each class for all data points(for which we will divide class by total classes) 
In a 'Random' Model, we generate the NINE class probabilites randomly such that they sum to 1.'''

In [ ]:
# for cross validation data

from sklearn.metrics.classification import accuracy_score, log_loss

cross_validation_size = y_cv.shape[0]

random_cross_validation = np.zeros((cross_validation_size,9)) # initializing with 0

for i in range(cross_validation_size):
    random_prob = np.random.rand(1,9)
    random_cross_validation[i] = ((random_prob/sum(sum(random_prob)))[0])
print("log loss for random cross validation is", log_loss(y_cv,random_cross_validation))    
  

In [ ]:
# for test data

from sklearn.metrics.classification import accuracy_score, log_loss

test_size = y_test.shape[0]

random_test_data = np.zeros((test_size,9)) # initializing with 0

for i in range(test_size):
    random_prob = np.random.rand(1,9)
    random_test_data[i] = ((random_prob/sum(sum(random_prob)))[0])
print("log loss for random test data is", log_loss(y_test,random_test_data))   

In [ ]:
'''by  both obervation we can say that thershold is approximately 2.5 which means we have to build our model 
having log loss less than 2.5'''

## Univariate analysis

### Gene analysis

In [ ]:
"""now we will do analysis of each feature one by one so lets start with gene"""

In [ ]:
unique_gene = result["Gene"].value_counts()
unique_gene.head(20)

In [ ]:
s = sum(unique_gene.values)
h = unique_gene/s
h

In [ ]:
f, ax = plt.subplots(figsize=(10,6))
plt.plot(h, label="Histrogram of Genes")
plt.xlabel('Index of a Gene')
plt.ylabel('Number of Occurances')
plt.legend()
plt.show()

In [ ]:
c = np.cumsum(h)
plt.plot(c,label='Cumulative distribution of Genes')
plt.grid()
plt.legend()
plt.show()

In [ ]:
x_train['Gene'].shape

In [ ]:
x_test['Gene'].shape

In [ ]:
x_cv['Gene'].shape

In [ ]:

# converting categorical data into numerisal data , we will use two menthods
# 1.One hot encoding
# 2.Response coding

# 1.One hot encoding

from sklearn.feature_extraction.text import CountVectorizer

gene_vectorizer = CountVectorizer()

train_gene_feature_onehotCoding = gene_vectorizer.fit_transform(x_train['Gene'])
test_gene_feature_onehotCoding = gene_vectorizer.transform(x_test['Gene'])
cv_gene_feature_onehotCoding = gene_vectorizer.transform(x_cv['Gene'])

In [ ]:
print(train_gene_feature_onehotCoding.shape)
print(test_gene_feature_onehotCoding.shape)
print(cv_gene_feature_onehotCoding.shape)

In [ ]:
print(train_gene_feature_onehotCoding.toarray())

In [ ]:
# apply SGD algorithm

from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV

alpha = [10**i for i in range(-5,1)]   # [1e-05, 0.0001, 0.001, 0.01, 0.1, 1]

cv_log_error_array=[]

for i in alpha:
    clf = SGDClassifier(loss='log' , alpha = i ,random_state = 42)
    clf.fit(train_gene_feature_onehotCoding , y_train)
    calibration = CalibratedClassifierCV(clf, method='sigmoid')
    calibration.fit(train_gene_feature_onehotCoding , y_train)
    y_cv_predict = calibration.predict_proba(cv_gene_feature_onehotCoding)
    cv_log_error_array.append(log_loss(y_cv, y_cv_predict))
    print('For values of alpha = ', i, "The log loss is:",log_loss(y_cv, y_cv_predict)) 
      

In [ ]:
print(cv_log_error_array)

In [ ]:

min_log_loss_position = np.argmin(cv_log_error_array)  # provide position of minimum value 


clf = SGDClassifier(loss='log' , alpha = alpha[min_log_loss_position],random_state = 42)

clf.fit(train_gene_feature_onehotCoding , y_train)
calibration = CalibratedClassifierCV(clf, method='sigmoid')

calibration.fit(train_gene_feature_onehotCoding , y_train)

y_train_predict = calibration.predict_proba(train_gene_feature_onehotCoding)
y_cv_predict = calibration.predict_proba(cv_gene_feature_onehotCoding)
y_test_predict = calibration.predict_proba(test_gene_feature_onehotCoding)

print('For values of alpha as',alpha[min_log_loss_position], "The log loss for train data is:",log_loss(y_train, y_train_predict)) 
print('For values of alpha as',alpha[min_log_loss_position], "The log loss for cv data is:",log_loss(y_cv, y_cv_predict)) 
print('For values of alpha as',alpha[min_log_loss_position], "The log loss for test data is:",log_loss(y_test, y_test_predict)) 


In [ ]:
'''now by looking at all the three log losses we can say that we have to keep gene feature as log loss is less by keeping
only this feature as compared to random log loss'''

### Variation analysis

In [ ]:
unique_variant = result["Variation"].value_counts()

In [ ]:
unique_variant

In [ ]:
h = unique_variant/sum(unique_variant)

In [ ]:
f, ax = plt.subplots(figsize=(10,6))
plt.plot(h, label="Histrogram of Variation")
plt.xlabel('Index of a Variation')
plt.ylabel('Number of Occurances')
plt.legend()
plt.show()

In [ ]:
# cdf

cdf = np.cumsum(h)
f, ax = plt.subplots(figsize=(10,6))
plt.plot(cdf, label="cdf of Variation")
plt.xlabel('Index of a Variation')
plt.ylabel('Number of Occurances')
plt.legend()
plt.show()

In [ ]:
# 1.One hot coding

from sklearn.feature_extraction.text import CountVectorizer

variation_vectorizer = CountVectorizer()

train_variation_feature_onehotCoding = variation_vectorizer.fit_transform(x_train['Variation'])
test_variation_feature_onehotCoding = variation_vectorizer.transform(x_test['Variation'])
cv_variation_feature_onehotCoding = variation_vectorizer.transform(x_cv['Variation'])

In [ ]:
print(train_variation_feature_onehotCoding)

In [ ]:
# apply SGD algorithm

from sklearn.calibration import CalibratedClassifierCV

alpha = [10**i for i in range(-5,1)]   # [1e-05, 0.0001, 0.001, 0.01, 0.1, 1]

cv_log_error_array=[]

for i in alpha:
    clf = SGDClassifier(loss='log' , alpha = i, random_state=42)
    clf.fit(train_variation_feature_onehotCoding , y_train)
    calibration = CalibratedClassifierCV(clf, method='sigmoid')
    calibration.fit(train_variation_feature_onehotCoding , y_train)
    y_cv_predict = calibration.predict_proba(cv_variation_feature_onehotCoding)
    cv_log_error_array.append(log_loss(y_cv, y_cv_predict))
    print('For values of alpha = ', i, "The log loss is:",log_loss(y_cv, y_cv_predict)) 

In [ ]:
min_log_loss_position = np.argmin(cv_log_error_array)  # provide position of minimum value 


clf = SGDClassifier(loss='log' , alpha = alpha[min_log_loss_position], random_state=42)

clf.fit(train_variation_feature_onehotCoding , y_train)
calibration = CalibratedClassifierCV(clf, method='sigmoid')
calibration.fit(train_variation_feature_onehotCoding , y_train)

y_train_predict = calibration.predict_proba(train_variation_feature_onehotCoding)
y_cv_predict = calibration.predict_proba(cv_variation_feature_onehotCoding)
y_test_predict = calibration.predict_proba(test_variation_feature_onehotCoding)

print('For values of alpha as',alpha[min_log_loss_position], "The log loss for train data is:",log_loss(y_train, y_train_predict)) 
print('For values of alpha as',alpha[min_log_loss_position], "The log loss for cv data is:",log_loss(y_cv, y_cv_predict)) 
print('For values of alpha as',alpha[min_log_loss_position], "The log loss for test data is:",log_loss(y_test, y_test_predict)) 


In [ ]:
'''now by looking at all the three log losseses we can say that we have to keep variation feature as log loss 
is less by keeping only this feature as compared to random log loss'''

### Text analysis

In [ ]:
# 1.One hot coding


from sklearn.feature_extraction.text import CountVectorizer

# building a CountVectorizer with all the words that occured minimum 3 times in train data
text_vectorizer = CountVectorizer(min_df=3)

train_text_feature_onehotCoding = text_vectorizer.fit_transform(x_train['TEXT'])

# getting all the feature names (words)
train_text_features= text_vectorizer.get_feature_names()

# train_text_feature_onehotCoding.sum(axis=0).A1 will sum every row and returns (1*number of features) vector
train_text_fea_counts = train_text_feature_onehotCoding.sum(axis=0).A1

# zip(list(text_features),text_fea_counts) will zip a word with its number of times it occured
text_fea_dict = dict(zip(list(train_text_features),train_text_fea_counts))


print("Total number of unique words in train data :", len(train_text_features))

In [ ]:
from sklearn.preprocessing import normalize

# don't forget to normalize every feature
train_text_feature_onehotCoding = normalize(train_text_feature_onehotCoding, axis=0)

# we use the same vectorizer that was trained on train data
test_text_feature_onehotCoding = text_vectorizer.transform(x_test['TEXT'])

# don't forget to normalize every feature
test_text_feature_onehotCoding = normalize(test_text_feature_onehotCoding, axis=0)

# we use the same vectorizer that was trained on train data
cv_text_feature_onehotCoding = text_vectorizer.transform(x_cv['TEXT'])

# don't forget to normalize every feature
cv_text_feature_onehotCoding = normalize(cv_text_feature_onehotCoding, axis=0)

In [ ]:
# apply SGD algorithm

from sklearn.calibration import CalibratedClassifierCV

alpha = [10**i for i in range(-5,1)]   # [1e-05, 0.0001, 0.001, 0.01, 0.1, 1]

cv_log_error_array=[]

for i in alpha:
    clf = SGDClassifier(loss='log' , alpha = i, random_state=42)
    clf.fit(train_text_feature_onehotCoding , y_train)
    calibration = CalibratedClassifierCV(clf, method='sigmoid')
    calibration.fit(train_text_feature_onehotCoding , y_train)
    y_cv_predict = calibration.predict_proba(cv_text_feature_onehotCoding)
    cv_log_error_array.append(log_loss(y_cv, y_cv_predict))
    print('For values of alpha = ', i, "The log loss is:",log_loss(y_cv, y_cv_predict))

In [ ]:
min_log_loss_position = np.argmin(cv_log_error_array)  # provide position of minimum value 


clf = SGDClassifier(loss='log' , alpha = alpha[min_log_loss_position], random_state=42)

clf.fit(train_text_feature_onehotCoding , y_train)
calibration = CalibratedClassifierCV(clf, method='sigmoid')
calibration.fit(train_text_feature_onehotCoding , y_train)

y_train_predict = calibration.predict_proba(train_text_feature_onehotCoding)
y_cv_predict = calibration.predict_proba(cv_text_feature_onehotCoding)
y_test_predict = calibration.predict_proba(test_text_feature_onehotCoding)

print('For values of alpha as',alpha[min_log_loss_position], "The log loss for train data is:",log_loss(y_train, y_train_predict)) 
print('For values of alpha as',alpha[min_log_loss_position], "The log loss for cv data is:",log_loss(y_cv, y_cv_predict)) 
print('For values of alpha as',alpha[min_log_loss_position], "The log loss for test data is:",log_loss(y_test, y_test_predict)) 


In [ ]:
'''now by looking at all the three log losseses we can say that we have to keep variation feature as log loss 
is less by keeping only this feature as compared to random log loss'''

# Applying Machine Learning Models

In [ ]:
# lets make a function for plotting confusion matrix(C), precision(B) and recall(A) matrices

from sklearn.metrics import confusion_matrix

def plot_mpr_matrices_heatmaps(y_actual,y_predicted):
    
    C = confusion_matrix(y_actual, y_predicted)
    # C = 9,9 matrix, each cell (i,j) represents number of points of class i are predicted class j
    
    A =(((C.T)/(C.sum(axis=1))).T)
    #divid each element of the confusion matrix with the sum of elements in that column
    
    # C = [[1, 2],
    #     [3, 4]]
    # C.T = [[1, 3],
    #        [2, 4]]
    # C.sum(axis = 1)  axis=0 corresonds to columns and axis=1 corresponds to rows in two diamensional array
    # C.sum(axix =1) = [[3, 7]]
    # ((C.T)/(C.sum(axis=1))) = [[1/3, 3/7]
    #                           [2/3, 4/7]]

    # ((C.T)/(C.sum(axis=1))).T = [[1/3, 2/3]
    #                           [3/7, 4/7]]
    # sum of row elements = 1
    
    B =(C/C.sum(axis=0))
    #divid each element of the confusion matrix with the sum of elements in that row
    # C = [[1, 2],
    #     [3, 4]]
    # C.sum(axis = 0)  axis=0 corresonds to columns and axis=1 corresponds to rows in two diamensional array
    # C.sum(axix =0) = [[4, 6]]
    # (C/C.sum(axis=0)) = [[1/4, 2/6],
    #                      [3/4, 4/6]] 
    
    labels = [1,2,3,4,5,6,7,8,9]
    
    print("-"*20, "Confusion matrix", "-"*20)
    
    plt.figure(figsize=(20,7))
    sns.heatmap(C, annot=True, cmap="YlGnBu", fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.show()

    print("-"*20, "Precision matrix (Columm Sum=1)", "-"*20)
    
    plt.figure(figsize=(20,7))
    sns.heatmap(B, annot=True, cmap="YlGnBu", fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.show()
    
    print("-"*20, "Recall matrix (Row sum=1)", "-"*20)
    
    plt.figure(figsize=(20,7))
    sns.heatmap(A, annot=True, cmap="YlGnBu", fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.show()
    
    
    

In [ ]:
# lets make a function for determining words cause of which our model is giving corresponding class value,
# we will use this fuction at the end of each algorithms.

#example:-Suppose our model give class label 6 for some test query, than what words our model found in that test query
#    which make it giving class label 6 for that 

def get_impfeature_names(indices, text, gene, var, no_features):
    gene_count_vec = CountVectorizer()
    var_count_vec = CountVectorizer()
    text_count_vec = CountVectorizer(min_df=3)
    
    gene_fea_count_vec = gene_count_vec.fit(x_train["Gene"])
    var_fea_count_vec = var_count_vec.fit(x_train["Variation"])
    text_fea_count_vec = text_count_vec.fit(x_train["TEXT"])
    
    len_unique_gene = len(gene_fea_count_vec.get_feature_names())  # 230
    len_unique_var = len(var_fea_count_vec.get_feature_names())   #1967
    
    word_count = 0
    for i , index in enumerate(indices):
        if index < len_unique_gene:
            word = gene_fea_count_vec.get_feature_names()[index]
            yes_no = True if word == gene else False
            if yes_no:
                word_count = word_count+1
                print(i, "Gene feature [{}] present in test data point [{}]".format(word,yes_no))   
            
        elif index < (len_unique_gene + len_unique_var):
            word = var_fea_count_vec.get_feature_names()[index - len_unique_gene]
            yes_no = True if word == var else False
            if yes_no:
                word_count = word_count+1
                print(i, "Var feature [{}] present in test data point [{}]".format(word,yes_no))  
                
        else:
            word = text_fea_count_vec.get_feature_names()[index - (len_unique_gene + len_unique_var)]
            yes_no = True if word in text.split() else False
            if yes_no:
                word_count = word_count+1
                print(i, "Text feature [{}] present in test data point [{}]".format(word,yes_no))   
                
    print("out of a total of",no_features,"feature",word_count,"are present in query point")         
                


In [ ]:
    gene_count_vec = CountVectorizer()
    var_count_vec = CountVectorizer()
    text_count_vec = CountVectorizer(min_df=3)
    
    gene_fea_count_vec = gene_count_vec.fit(x_train["Gene"])
    var_fea_count_vec = var_count_vec.fit(x_train["Variation"])
    text_fea_count_vec = text_count_vec.fit(x_train["TEXT"])


len(var_fea_count_vec.get_feature_names())

In [ ]:
#after encoding all the 3 features we have to stack them together so as to make our complete datasets[D{xi,yi}]

import numpy as np
from scipy.sparse import hstack


# final datasets with one hot encoding


#stacking gene and variation
train_gene_var_onehotCoding = hstack((train_gene_feature_onehotCoding,train_variation_feature_onehotCoding))
test_gene_var_onehotCoding = hstack((test_gene_feature_onehotCoding,test_variation_feature_onehotCoding))
cv_gene_var_onehotCoding = hstack((cv_gene_feature_onehotCoding,cv_variation_feature_onehotCoding))


# train data
train_x_onehotCoding = hstack((train_gene_var_onehotCoding, train_text_feature_onehotCoding)).tocsr()
train_y = np.array(list(y_train['Class']))

# test data
test_x_onehotCoding = hstack((test_gene_var_onehotCoding, test_text_feature_onehotCoding)).tocsr()
test_y = np.array(list(y_test['Class']))

#cv data
cv_x_onehotCoding = hstack((cv_gene_var_onehotCoding, cv_text_feature_onehotCoding)).tocsr()
cv_y = np.array(list(y_cv['Class']))




In [ ]:
print("One hot encoding features :\n")

print("(number of data points * number of features) in train data = ", train_x_onehotCoding.shape)
print("(number of data points * number of features) in test data = ", test_x_onehotCoding.shape)
print("(number of data points * number of features) in cross validation data =", cv_x_onehotCoding.shape)

# 1. Naive Bayes

In [ ]:
# As we are doing multi classification problem so we have to use multinomial naive bayes here .
# here hyper parameter is alpha which is laplace smoothing

from sklearn.naive_bayes import MultinomialNB

# finding best hyper parameter

alpha = [10**x for x in range(-5,3)]

cv_log_error_array = []

for i in alpha:
    clf = MultinomialNB(alpha = i)
    clf.fit(train_x_onehotCoding , train_y)
    calibration = CalibratedClassifierCV(clf , method='sigmoid')
    calibration.fit(train_x_onehotCoding , train_y)
    y_pred = calibration.predict_proba(cv_x_onehotCoding)
    cv_log_error_array.append(log_loss(cv_y , y_pred))
    print("log_loss for alpha",i,"is",log_loss(cv_y , y_pred))
    

In [ ]:
# plotting between alpha and log loss to see at what alpha log loss is minimum (same thing as above in graph)

f, ax = plt.subplots(1,figsize=(10,5))
ax.plot(np.log10(alpha), cv_log_error_array)
for i, txt in enumerate(np.round(cv_log_error_array,3)):
    ax.annotate((alpha[i],str(txt)), (np.log10(alpha[i]),cv_log_error_array[i]))
plt.grid()
plt.xticks(np.log10(alpha))
plt.title("Cross Validation Error for each alpha")
plt.xlabel("Alpha i's")
plt.ylabel("Error")
plt.show()


In [ ]:
# using best hyper parameter for our model

best_alpha = np.argmin(cv_log_error_array)
clf = MultinomialNB(alpha=alpha[best_alpha])
clf.fit(train_x_onehotCoding, train_y)
sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
sig_clf.fit(train_x_onehotCoding, train_y)


predict_y_train = sig_clf.predict_proba(train_x_onehotCoding)
predict_y_cv = sig_clf.predict_proba(cv_x_onehotCoding)
predict_y_test = sig_clf.predict_proba(test_x_onehotCoding)


print('For values of best alpha = ', alpha[best_alpha], "The train log loss is:",log_loss(train_y, predict_y_train))
predict_y = sig_clf.predict_proba(cv_x_onehotCoding)
print('For values of best alpha = ', alpha[best_alpha], "The cross validation log loss is:",log_loss(cv_y, predict_y_cv))
predict_y = sig_clf.predict_proba(test_x_onehotCoding)
print('For values of best alpha = ', alpha[best_alpha], "The test log loss is:",log_loss(test_y, predict_y_test,))


 <h5>As we understand log losses for all data, now we will make confusion matrix,precicion matrix,recall matrix for cv data to
understand better that how misclassification happening between classes.</h5>

In [ ]:

clf = MultinomialNB(alpha=alpha[best_alpha])
clf.fit(train_x_onehotCoding, train_y)
sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
sig_clf.fit(train_x_onehotCoding, train_y)

sig_clf_predict_probs = sig_clf.predict_proba(cv_x_onehotCoding)

sig_clf_predict = sig_clf.predict(cv_x_onehotCoding)

print("Log Loss cv data:",log_loss(cv_y, sig_clf_predict_probs))

print("Number of missclassified point is", np.count_nonzero((sig_clf_predict - cv_y)),"out of total",cv_y.shape[0],"points")

# we are using the function that we declared earlier for plotting mpr matrices

plot_mpr_matrices_heatmaps(cv_y,sig_clf_predict)

### Last and main thing

In [ ]:
# lets see how our model works for a query test data

# we will see for first(first index) test data and lets see whether predicted and actual same or not and also find array of 
# probablities to it as it is our business requirement.

test_point_index = 6
no_feature = 100


query_data = test_x_onehotCoding[test_point_index]

predicted_class = sig_clf.predict(query_data)

predicted_probablity = sig_clf.predict_proba(query_data)

print("Our model predicted class of this query data is =" ,predicted_class[0])

print("Our model predicted probablities for all classes for this query data is =" ,np.round(predicted_probablity,3))

print("Actual class of this query data is =" , test_y[test_point_index])


# now the main thing is, for interpretability we will find words cause of which corresponding class predicted 
# feature importance


# calculating indices of datas having class same as our predicted class
indices = np.argsort(-1*abs(clf.coef_))[predicted_class-1][:,:no_feature]

print("-"*100)

get_impfeature_names(indices[0], x_test['TEXT'].iloc[test_point_index],x_test['Gene'].iloc[test_point_index],
                     x_test['Variation'].iloc[test_point_index], no_feature)



# 2. K-Nearest Neighbour

In [ ]:
# as we know that knn wont work for high dimension data cause of curse of dimensionality ,but still lets see.

from sklearn.neighbors import KNeighborsClassifier

k = [x for x in range(1,20,2)]

cv_log_loss_array = []

for i in k:
    
    clf = KNeighborsClassifier(n_neighbors = i)
    clf.fit(train_x_onehotCoding , train_y)
    calibration = CalibratedClassifierCV( clf , method='sigmoid')
    calibration.fit(train_x_onehotCoding , train_y)
    pred_probablity = calibration.predict_proba(cv_x_onehotCoding)
    cv_log_loss_array.append(log_loss(cv_y , pred_probablity))
    print("log_loss for k = ",i,"is",log_loss(cv_y , pred_probablity))

In [ ]:
# plotting for k vs error

f,ax = plt.subplots(1 , figsize = (15,5))
ax.plot(k , cv_log_loss_array)
for i,text in enumerate (np.round(cv_log_loss_array , 3)) :
    ax.annotate((k[i] , str(text)) , (k[i] , cv_log_loss_array[i] ))
    
plt.grid()
plt.title("Cross Validation Error for each alpha")
plt.xlabel("K's")
plt.ylabel("Error measure")
plt.show()

In [ ]:
# finding log loss for the best hyperparameter

best_k_position = np.argmin(cv_log_loss_array)


clf = KNeighborsClassifier(n_neighbors = k[best_k_position])
clf.fit(train_x_onehotCoding , train_y)
calibration = CalibratedClassifierCV( clf , method='sigmoid')
calibration.fit(train_x_onehotCoding , train_y)
clf_predict = calibration.predict(cv_x_onehotCoding)


predict_y_train = calibration.predict_proba(train_x_onehotCoding)
predict_y_cv = calibration.predict_proba(cv_x_onehotCoding)
predict_y_test = calibration.predict_proba(test_x_onehotCoding)


print('For values of best k = ', k[best_k_position], "The train log loss is:",log_loss(train_y, predict_y_train))

print('For values of best k = ', k[best_k_position], "The cross validation log loss is:",log_loss(cv_y, predict_y_cv))

print('For values of best k = ', k[best_k_position], "The test log loss is:",log_loss(test_y, predict_y_test,))


In [ ]:
# we are using the function that we declared earlier for plotting mpr matrices and also we will find total misclassified 
# points ,both thing for our cv data.

total_misclassified_point = np.count_nonzero(clf_predict-cv_y)
print("There are" , total_misclassified_point,"misclassified points out of a total of", cv_y.shape[0],"points\n")

plot_mpr_matrices_heatmaps(cv_y,clf_predict)

### Last and main thing

In [ ]:
# lets see how our model works for a query test data

# we will see for first(first index) test data and lets see whether predicted and actual same or not and also find array of 
# probablities to it as it is our business requirement.
# we have to reshape here at last as we are using response coding

from sklearn import neighbors

clf = KNeighborsClassifier(n_neighbors = k[best_k_position])
clf.fit(train_x_onehotCoding , train_y)
calibration = CalibratedClassifierCV( clf , method='sigmoid')
calibration.fit(train_x_onehotCoding , train_y)

test_point_index = 5
no_feature = 100



query_data = test_x_onehotCoding[test_point_index]

predicted_class = calibration.predict(test_x_onehotCoding[test_point_index].reshape(1,-1))

predicted_class_probablity = calibration.predict_proba(test_x_onehotCoding[test_point_index].reshape(1,-1))

print("Our model predicted class of this query data is =" , predicted_class[0])

print("Our model predicted probablities for all classes for this query data is =" ,np.round(predicted_class_probablity,3))

print("Actual class of this query data is =" , test_y[test_point_index])

neighbors = clf.kneighbors(test_x_onehotCoding[test_point_index].reshape(1, -1), k[best_k_position])

print("The ",k[best_k_position]," nearest neighbours of the query point belongs to classes",train_y[neighbors[1][0]])


# 3. Logistic Regression

In [ ]:
# now we will make model by logistic regression with two types of data ,first one is with balanced data and the other
# one is imbalanced data.
# we will use logistiv=c regression by selecting sgd classfier and in argumnent selecting loss as log

### Logistic Regression with balanced data

In [ ]:
alpha = [10**x for x in range(-5,3)]        # hyperparameter

cv_log_loss_array = []

for i in alpha:
    
    clf = SGDClassifier(loss='log',class_weight = 'balanced',alpha = i , random_state=42)   # balancing weight
    clf.fit(train_x_onehotCoding , train_y)
    calibration = CalibratedClassifierCV(base_estimator = clf)
    calibration.fit(train_x_onehotCoding , train_y)
    pred_probality = calibration.predict_proba(cv_x_onehotCoding)
    cv_log_loss_array.append(log_loss(cv_y , pred_probality))
    print("log loss for alpha value",i,"is",log_loss(cv_y , pred_probality))
    

In [ ]:
# plotting error vs hyperparamter

fig,ax = plt.subplots(figsize = (10,8))

ax.plot(alpha , cv_log_loss_array )

for i,text in enumerate(np.round(cv_log_loss_array,3)):
    ax.annotate((alpha[i] , text) ,(alpha[i] ,cv_log_loss_array[i]))
    
plt.grid()
plt.title("Cross Validation Error for each alpha")
plt.xlabel("Alpha i's")
plt.ylabel("Error measure")
plt.show()    

In [ ]:
# finding log loss for best hyperparmeter

best_alpha_position = np.argmin(cv_log_loss_array)


clf = SGDClassifier(loss='log',class_weight = 'balanced', alpha = alpha[best_alpha_position] , random_state=42) 
clf.fit(train_x_onehotCoding , train_y)
calibration = CalibratedClassifierCV(base_estimator = clf)
calibration.fit(train_x_onehotCoding , train_y)

pred_cv = calibration.predict(cv_x_onehotCoding)


pred_probality_train = calibration.predict_proba(train_x_onehotCoding)
pred_probality_cv = calibration.predict_proba(cv_x_onehotCoding)
pred_probality_test = calibration.predict_proba(test_x_onehotCoding)


print("train data log loss for alpha =",alpha[best_alpha_position],"is",log_loss(train_y , pred_probality_train))
print("cv data log loss for alpha =",alpha[best_alpha_position],"is",log_loss(cv_y , pred_probality_cv))
print("test data log loss for alpha =",alpha[best_alpha_position],"is",log_loss(test_y , pred_probality_test))


In [ ]:
# misclassfied points and plotting matrices

total_misclassified_point = np.count_nonzero(cv_y - pred_cv)

print("total misclassfied point is",total_misclassified_point,"out of a total of",cv_y.shape[0],"data points")



In [ ]:
plot_mpr_matrices_heatmaps(cv_y,pred_cv)

In [ ]:
# now testing our model with a query and we will also use a function to calculate its feature importance also.

clf = SGDClassifier(alpha=alpha[best_alpha], penalty='l2', loss='log', random_state=42)
clf.fit(train_x_onehotCoding,train_y)
test_point_index = 0
no_feature = 500

predicted_cls = calibration.predict(test_x_onehotCoding[test_point_index])

print("Predicted Class :", predicted_cls[test_point_index])
print("Predicted Class Probabilities:", np.round(sig_clf.predict_proba(test_x_onehotCoding[test_point_index]),4))
print("Actual Class :", test_y[test_point_index])


In [ ]:
# to calculate feature importance

# calculating indices of datas having class same as our predicted class
indices = np.argsort(-1*abs(clf.coef_))[predicted_cls-1][:,:no_feature]

get_impfeature_names(indices[0], x_test['TEXT'].iloc[test_point_index],x_test['Gene'].iloc[test_point_index],
                     x_test['Variation'].iloc[test_point_index], no_feature)

### Logistic Regression with imbalanced data

In [ ]:
# now for class imbalanced dataset for logistice regression we just have to change in argument of weight balancing
# rest everything will be same .

# 4. Linear SVM

In [ ]:
# we will use linear svm as it is same as logistice regression and also works very good on high dimension data
# we are not using rbf svm as we donot know the correct kernel for it ,also it is not interpretable.

alpha = [10**x for x in range(-5,4)]  # hyperparameter

cv_log_error = []

for i in alpha:
    clf = SGDClassifier(loss='hinge', penalty='l2', alpha = i , class_weight = 'balanced',random_state=42)
    clf.fit(train_x_onehotCoding , train_y)
    calibration_svm = CalibratedClassifierCV(base_estimator = clf ,  method='sigmoid')
    calibration_svm.fit(train_x_onehotCoding , train_y)
    pred_probality_cv = calibration_svm.predict_proba(cv_x_onehotCoding)
    cv_log_error.append(log_loss(cv_y , pred_probality_cv))
    print("log loss for cv data for svm model with alpha as",i,"is",log_loss(cv_y , pred_probality_cv))

In [ ]:
# plotting error vs hyperparameter

fig,ax = plt.subplots(figsize = (20,15))

ax.plot(alpha , cv_log_error )

for i,text in enumerate(np.round(cv_log_error,3)):
    ax.annotate((alpha[i] , text) ,(alpha[i] ,cv_log_error[i]))
    
plt.grid()
plt.title("Cross Validation Error for each alpha")
plt.xlabel("Alpha i's")
plt.ylabel("Error measure")
plt.show()  

In [ ]:
# now lets start for best hyper parameter

best_alpha_position = np.argmin(cv_log_error)

clf = SGDClassifier(loss='hinge', penalty='l2', alpha = alpha[best_alpha_position] , class_weight = 'balanced',random_state=42)
clf.fit(train_x_onehotCoding , train_y)
calibration_svm = CalibratedClassifierCV(base_estimator = clf ,  method='sigmoid')
calibration_svm.fit(train_x_onehotCoding , train_y)

pred_test_svm = calibration_svm.predict(test_x_onehotCoding)


pred_probality_train = calibration_svm.predict_proba(train_x_onehotCoding)
pred_probality_cv = calibration_svm.predict_proba(cv_x_onehotCoding)
pred_probality_test = calibration_svm.predict_proba(test_x_onehotCoding)



print("log loss for train data for svm model is",log_loss(train_y , pred_probality_train))
print("log loss for cv data for svm model is",log_loss(cv_y , pred_probality_cv))
print("log loss for test data for svm model is",log_loss(test_y , pred_probality_test))



In [ ]:
# total misclassified points by svm and its confusion matrices

pred_cv_svm = calibration_svm.predict(cv_x_onehotCoding)
total_misclassfied_svm = np.count_nonzero(cv_y - pred_cv_svm )

print("total misclassfied poinnts for svm model is",total_misclassfied_svm,"out of",cv_y.shape[0],"data points")


In [ ]:
plot_mpr_matrices_heatmaps(cv_y,pred_cv_svm)

In [ ]:
# now testing our model with query test point and find feature importance


test_point_index = 0
no_of_feature  = 1000

print("predicted value for given  test data is ",pred_test_svm[test_point_index])
print("actual value for given test data is ",test_y[test_point_index])
print("predicted probablistic value for given test data is ",np.round(pred_probality_test[test_point_index],3))


In [ ]:
# feature importance

indices = np.argsort(-1*abs(clf.coef_))[pred_test_svm-1][:,:no_of_feature]

get_impfeature_names(indices[0], x_test['TEXT'].iloc[test_point_index],x_test['Gene'].iloc[test_point_index],
                     x_test['Variation'].iloc[test_point_index], no_of_feature)

# 5. Random Forest Classifier

In [ ]:
# as rf depends on decision trees so less dimension data set will be better but still we will use both response coded 
# and one hot encoded data set to check the model.

# In rf ,there are two hyperparmeters first one is total trees(n_estimators) and second is max depth of each tree. 

### Random forest (one hot encoded)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

max_depth = [5,10,15]
max_trees = [x for x in range(100,1000,200)]


cv_log_rf_error = []
for i in max_trees:
    for j in max_depth:
        clf = RandomForestClassifier(n_estimators = i , criterion='gini', max_depth = j ,random_state = 42)
        clf.fit(train_x_onehotCoding , train_y)
        calibration_rf = CalibratedClassifierCV(base_estimator = clf)
        calibration_rf.fit(train_x_onehotCoding , train_y)
        predict_cv_class = calibration_rf.predict_proba(cv_x_onehotCoding)
        cv_log_rf_error.append(log_loss(cv_y , predict_cv_class))
        print("log loss for max_trees as",i,"and max_depth as",j,"is",log_loss(cv_y , predict_cv_class))
        
        

In [ ]:
min_log_loss_position = np.argmin(cv_log_rf_error)
min_log_loss_position

In [ ]:
min_log_loss_position = np.argmin(cv_log_rf_error)
min_log_loss_position
# now for best hyperparameter calculating log loss in all data

clf = RandomForestClassifier(n_estimators = 900 , criterion='gini', max_depth = 10 ,random_state = 42)
clf.fit(train_x_onehotCoding , train_y)
calibration_rf = CalibratedClassifierCV(base_estimator = clf)
calibration_rf.fit(train_x_onehotCoding , train_y)

predict_cv_class = calibration_rf.predict(cv_x_onehotCoding)

predict_prob__train_class = calibration_rf.predict_proba(train_x_onehotCoding)
predict_prob_cv_class = calibration_rf.predict_proba(cv_x_onehotCoding)
predict_prob_test_class = calibration_rf.predict_proba(test_x_onehotCoding)


print("log loss for best hyperparameter for train data is",log_loss(train_y , predict_prob__train_class))
print("log loss for best hyperparameter for cv data is",log_loss(cv_y , predict_prob_cv_class))
print("log loss for best hyperparameter for test data is",log_loss(test_y , predict_prob_test_class))


In [ ]:
# calculating misclassified points and plotting metrices

total_misclassified_rf = np.count_nonzero(cv_y - predict_cv_class)

print("total misclassified point is",total_misclassified_rf,"out of a total of",cv_y.shape[0],"points")


In [ ]:
plot_mpr_matrices_heatmaps(cv_y,predict_cv_class)

In [ ]:
# testing model with a test data and calculating feature importance

test_point_index = 0
no_of_feature  = 100

pred_test_rf = calibration_rf.predict(test_x_onehotCoding)

print("predicted value for given  test data is ",pred_test_rf[test_point_index])
print("actual value for given test data is ",test_y[test_point_index])
print("predicted probablistic value for given test data is ",np.round(predict_prob_test_class[test_point_index],3))



# 6. Stacking model

For this we will use two types of classfier\
a) Stacking classfier\
b) Voting classifier

Problem with stacking model is

1.Interpretability lost(no feature importance).\
2.Data separted between different classfier so individual accuracy decreases.\
3.Stacking model is good when we have millions of data .

## Stacking Classifier

In [ ]:
# we will use three different classifier and then merge it to the single classifier(default = logistic regression) 

# logistic regression
clf_1 = SGDClassifier(alpha=0.001, penalty='l2', loss='log', class_weight='balanced', random_state=0)
clf_1.fit(train_x_onehotCoding, train_y)
sig_clf_1 = CalibratedClassifierCV(clf_1, method="sigmoid")

# svm
clf_2 = SGDClassifier(alpha=1, penalty='l2', loss='hinge', class_weight='balanced', random_state=0)
clf_2.fit(train_x_onehotCoding, train_y)
sig_clf_2 = CalibratedClassifierCV(clf_2, method="sigmoid")

# Naive Bayes

clf_3 = MultinomialNB(alpha=0.001)
clf_3.fit(train_x_onehotCoding, train_y)
sig_clf_3 = CalibratedClassifierCV(clf_3, method="sigmoid")


# log loss individual
sig_clf_1.fit(train_x_onehotCoding, train_y)
print("log loss for logistic regression is",log_loss(cv_y , sig_clf_1.predict_proba(cv_x_onehotCoding)))

sig_clf_2.fit(train_x_onehotCoding, train_y)
print("log loss for svm is",log_loss(cv_y , sig_clf_2.predict_proba(cv_x_onehotCoding)))

sig_clf_3.fit(train_x_onehotCoding, train_y)
print("log loss for naive bayes is",log_loss(cv_y , sig_clf_3.predict_proba(cv_x_onehotCoding)))

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

alpha = [10**x for x in range(-3,2)]

for i in alpha:
    final_classifier = LogisticRegression(C=i)
    sclf = StackingClassifier(estimators=[('lr', sig_clf_1), ('svc', sig_clf_2), ('nb', sig_clf_3)], final_estimator = final_classifier)
    sclf.fit(train_x_onehotCoding, train_y)
    predicted_prob_value = sclf.predict_proba(cv_x_onehotCoding)
    predicted_value = sclf.predict(cv_x_onehotCoding)
    
    print("log loss for the stacked classifier for alpha as",i,"is",log_loss(cv_y , predicted_prob_value))

In [ ]:
# testing model with best hyperparameter

final_classifier = LogisticRegression(C=1)
sclf = StackingClassifier(estimators=[('lr', sig_clf_1), ('svc', sig_clf_2), ('nb', sig_clf_3)], final_estimator = final_classifier)
sclf.fit(train_x_onehotCoding, train_y)

log_error = log_loss(train_y, sclf.predict_proba(train_x_onehotCoding))
print("Log loss (train) on the stacking classifier :",log_error)

log_error = log_loss(cv_y, sclf.predict_proba(cv_x_onehotCoding))
print("Log loss (CV) on the stacking classifier :",log_error)

log_error = log_loss(test_y, sclf.predict_proba(test_x_onehotCoding))
print("Log loss (test) on the stacking classifier :",log_error)

print("Number of missclassified point :", np.count_nonzero((sclf.predict(test_x_onehotCoding)- test_y))/test_y.shape[0])
plot_mpr_matrices_heatmaps(cv_y , predicted_value)


## Maximun Voting classifier

In [ ]:
from sklearn.ensemble import VotingClassifier
vclf = VotingClassifier(estimators=[('lr', sig_clf_1), ('svc', sig_clf_2), ('nb', sig_clf_3)], voting='soft')
vclf.fit(train_x_onehotCoding, train_y)

predicted_value = vclf.predict(cv_x_onehotCoding)
print("Log loss (train) on the VotingClassifier :", log_loss(train_y, vclf.predict_proba(train_x_onehotCoding)))
print("Log loss (CV) on the VotingClassifier :", log_loss(cv_y, vclf.predict_proba(cv_x_onehotCoding)))
print("Log loss (test) on the VotingClassifier :", log_loss(test_y, vclf.predict_proba(test_x_onehotCoding)))
print("Number of missclassified point :", np.count_nonzero((vclf.predict(test_x_onehotCoding)- test_y))/test_y.shape[0])
plot_mpr_matrices_heatmaps(cv_y , predicted_value)